In [1]:
import json
import ollama
from ollama import Options


def extract_question_from_html(html_content):
    response = ollama.generate(
        model='gemma3:27b',
        prompt=f'''
             You are an expert in extracting exam questions from HTML content.
            
             I will provide you with HTML content containing an exam question and its multiple-choice options.
             Your task is to extract the question text and the options as separate items.
            
             HTML content:
             ```
             {html_content}
             ```
            
             Extract and return ONLY in the following JSON format:
             {{
                 "question": "The full question text here without any HTML tags",
                 "options": ["Option 1", "Option 2", "Option 3", "Option 4", "Option 5"]
             }}
            
             Guidelines:
             - Extract the complete question text, including any context or paragraphs preceding the actual question
             - Extract all options exactly as they appear
             - Return valid JSON only, nothing else
             - Remove any HTML tags from the question text and options
             - If you cannot find a question or options, return an empty question or options array
             - Do not include option letters (A, B, C, etc.) in the option text unless they are part of the actual content
             ''',
        options=Options(seed=42)
    )
    result = response['response']
    # Find the pattern that looks like valid JSON using string manipulation
    start_idx = result.find('{')
    end_idx = result.rfind('}') + 1
    if start_idx >= 0 and end_idx > start_idx:
        json_str = result[start_idx:end_idx]
        parsed_json = json.loads(json_str)
        return parsed_json
    raise ValueError("Could not extract valid question and options from response")



In [2]:
filePaths = [
    
    "biyoloji_6_questions.json",
]

In [3]:

stats = {}

for filename in filePaths:
    with open("questions/" + filename, 'r', encoding='utf-8') as f:
        questions = json.load(f)

        processedQuestions = [q for q in questions if 'answer' in q and q['answer'] < 6 and 'img' not in q['questionAsHtml'].lower()]
        
        # Initialize counters
        image_count = 0
        no_image_count = 0
        total_questions = len(processedQuestions)
        
        for question in processedQuestions:
            print(f"processing question: {processedQuestions.index(question)}/{len(processedQuestions)} id: {question['id']}")
            
            # Skip if question already has options
            if 'options' in question:
                continue
                
            has_image = 'questionAsHtml' in question and 'img' in question['questionAsHtml'].lower()
            hasMoreOptionThan5 = 'answer' in question and question['answer'] > 5
            
            if has_image:
                image_count += 1
            else:
                max_retries = 3
                retry_count = 0
                while retry_count < max_retries:
                    try:
                        # Add timeout of 60 seconds
                        import signal
                        def timeout_handler(signum, frame):
                            raise TimeoutError("Processing took too long")
                            
                        signal.signal(signal.SIGALRM, timeout_handler)
                        signal.alarm(60)  # Set 60 second timeout
                        
                        try:
                            questionResponse = extract_question_from_html(question['questionAsHtml'])
                            signal.alarm(0)  # Disable alarm
                            
                            if questionResponse is not None:
                                print(f"questionResponse: {questionResponse}")
                                question['question'] = questionResponse['question']
                                question['options'] = questionResponse['options']
                                
                                # Save after each successful question processing
                                with open("questions/" + filename, 'w', encoding='utf-8') as f:
                                    json.dump(processedQuestions, f, ensure_ascii=False, indent=2)
                                    
                                break
                            else:
                                print(f"Failed attempt {retry_count + 1} of {max_retries}")
                                retry_count += 1
                                
                        except TimeoutError:
                            print("Question processing timed out after 60 seconds")
                            retry_count += 1
                            signal.alarm(0)  # Disable alarm
                            
                    except Exception as e:
                        print(f"Error on attempt {retry_count + 1}: {str(e)}")
                        retry_count += 1
                        if retry_count == max_retries: 
                            print(f"Failed to process question after {max_retries} attempts")
                 
        # Add stats to dict if there are any href tags
        if image_count > 0:
            stats[filename] = { 
                'total_questions': total_questions,
                'image_count': image_count,
                'no_image_count': no_image_count
            }

processing question: 0/264 id: 2582
processing question: 1/264 id: 2583
processing question: 2/264 id: 2584
processing question: 3/264 id: 2585
processing question: 4/264 id: 2600
processing question: 5/264 id: 2608
processing question: 6/264 id: 2639
processing question: 7/264 id: 2654
processing question: 8/264 id: 2661
processing question: 9/264 id: 2662
processing question: 10/264 id: 2703
processing question: 11/264 id: 28875
processing question: 12/264 id: 28877
processing question: 13/264 id: 28893
processing question: 14/264 id: 28897
processing question: 15/264 id: 28903
processing question: 16/264 id: 28905
processing question: 17/264 id: 28908
processing question: 18/264 id: 28910
processing question: 19/264 id: 2586
processing question: 20/264 id: 2587
processing question: 21/264 id: 2588
processing question: 22/264 id: 2589
processing question: 23/264 id: 2591
processing question: 24/264 id: 2592
processing question: 25/264 id: 2593
processing question: 26/264 id: 2594
pro